### Fontes: 
- https://portaldatransparencia.gov.br/download-de-dados/licitacoes
- https://compras.dados.gov.br/licitacoes/v1/orgaos.json
- https://compras.dados.gov.br/licitacoes/v1/modalidades_licitacao.json

## 1) Bibliotecas

In [1]:
import pandas as pd
import requests
from sqlalchemy import create_engine
import psycopg2
from urllib.parse import urlparse
import os
import numpy as np

In [2]:
# Configurações do PostgreSQL
DB_HOST = "localhost"  
DB_PORT = "5432"       
DB_NAME = "licitacoes_contratos"  
DB_USER = "postgres"
DB_PASS = "1234"

# Outras configuracoes
diretorio = "Dados"

## 2) Funções

In [3]:
def verificar_tamanho_colunas(df):
    """
    Verifica o tamanho máximo dos valores em cada coluna de um DataFrame.

    Parâmetros:
    - df (pd.DataFrame): DataFrame a ser analisado.

    Retorna:
    - pd.DataFrame: DataFrame com o nome da coluna e o tamanho máximo dos valores.
    """
    tamanhos = {col: df[col].astype(str).map(len).max() for col in df.columns}
    df_tamanhos = pd.DataFrame(list(tamanhos.items()), columns=['Coluna', 'Tamanho Máximo'])
    
    return df_tamanhos

In [4]:
def ler_csv(arquivo, encoding="ISO-8859-1"):
    """
    Lê um arquivo CSV de um diretório específico e retorna um DataFrame.
    
    Parâmetros:
        nome_arquivo (str): Nome do arquivo CSV (ex: "202401_Licitação.csv").
        diretorio (str): Diretório onde o arquivo está localizado (padrão: "Dados").
    
    Retorna:
        pd.DataFrame: DataFrame contendo os dados do CSV.
    """
     
    # Ler o CSV com delimitador ";" e codificação ISO-8859-1
    df = pd.read_csv(arquivo, delimiter=";", encoding="ISO-8859-1", low_memory=False)
    return df


In [5]:
def ler_arquivo_csv(arquivo, encoding="ISO-8859-1"):

    df_licitacao = ler_csv(arquivo, encoding)
    
    # Extraindo ano e mês do nome do arquivo
    nome_arquivo = os.path.basename(arquivo)
    ano = nome_arquivo[:4]
    mes = nome_arquivo[4:6]
    
    # Adicionando colunas de ano e mês
    df_licitacao['ano'] = ano
    df_licitacao['mes'] = mes
    
    return df_licitacao


In [6]:
# Função para processar o diretório e carregar os arquivos CSV
def processar_diretorio(diretorio, chave, encoding="ISO-8859-1"):
    """
    Processa um diretório e concatena arquivos CSV que terminam com '_chave.csv'.

    Parâmetros:
    - diretorio (str): Caminho do diretório onde estão os arquivos CSV.
    - chave (str): Identificador para selecionar arquivos que terminam com '_chave.csv'.
    - encoding (str, opcional): Encoding usado para ler os arquivos (padrão: 'ISO-8859-1').

    Retorna:
    - pd.DataFrame: DataFrame contendo a concatenação de todos os arquivos encontrados.
    """
    # Listar arquivos que terminam com "_chave.csv"
    arquivos = [os.path.join(diretorio, f) for f in os.listdir(diretorio) if f.endswith(f"_{chave}.csv")]

    if not arquivos:
        print(f"⚠️ Nenhum arquivo encontrado em '{diretorio}' com a chave '{chave}'.")
        return pd.DataFrame()  # Retorna um DataFrame vazio se não houver arquivos

    # Passar corretamente os arquivos para a função `ler_arquivo_csv`
    dataframes = [ler_arquivo_csv(arquivo, encoding) for arquivo in arquivos]

    # Remover arquivos que falharam ao carregar (None)
    dataframes = [df for df in dataframes if df is not None]

    if not dataframes:
        print("❌ Nenhum arquivo válido foi carregado.")
        return pd.DataFrame()

    df_final = pd.concat(dataframes, ignore_index=True)

    print(f"✅ {len(dataframes)} arquivos processados e {df_final.shape[0]} registros carregados.")
    return df_final


In [7]:
def LerJson(aURL, aitem):
    
    try:
        
        erro = False
        df_link = pd.DataFrame()
        df_ret= pd.DataFrame()
        response = requests.get(aURL, timeout=5)
    
        if response.status_code == 200:

            dictr = response.json()

            # Recupera o nome da próxima pagina
            recs = dictr['_links']
            df_link = pd.json_normalize(recs)

            # Recupera os contratos
            recs = dictr['_embedded']
            df_ret = pd.json_normalize(recs, record_path=[aitem])
            
        else:
             erro  = True

        response.close()
        
    except Exception as e:
    # do whatever you want
        erro  = True
        
    return [df_link, df_ret, erro]


In [8]:
def obter_dados_json(url, chave):

    parsed_url = urlparse(url)
    url_base = f"{parsed_url.scheme}://{parsed_url.netloc}"

    df_final = pd.DataFrame()
    df_dados = pd.DataFrame()
    
    ret = LerJson(url, chave)
    df_pagina = ret[0]
    df_dados = pd.concat([df_dados, ret[1]])

    contador = 0
    maximo = 100
    while (contador < maximo):
        
        if df_pagina.shape[0] == 1:
            if df_pagina.columns.isin(['next.href']).any():
                print(url_base + df_pagina.loc[0,'next.href'])
                ret = LerJson(url_base + df_pagina.loc[0,'next.href'], chave) 
                if ret[2] == False:
                    df_pagina = ret[0]
                    df_dados = pd.concat([df_dados, ret[1]])
            else:
                break
        else:
            break
        contador = contador + 1
        
    return df_dados


In [9]:
def inserir_dados_postgresql(df, tabela, host, porta, banco, usuario, senha, excluir_antes=False, batch_size=10000):
    """
    Insere registros de um DataFrame no PostgreSQL em lotes de 1000 registros.

    Parâmetros:
    - df (pd.DataFrame): DataFrame com os dados a serem inseridos.
    - tabela (str): Nome da tabela de destino no PostgreSQL.
    - host (str): Endereço do servidor PostgreSQL.
    - porta (str): Porta do PostgreSQL.
    - banco (str): Nome do banco de dados.
    - usuario (str): Usuário do banco de dados.
    - senha (str): Senha do banco de dados.
    - excluir_antes (bool, opcional): Se True, exclui todos os registros antes da inserção (default: False).
    - batch_size (int, opcional): Número de registros inseridos por vez (default: 1000).

    Retorna:
    - str: Mensagem indicando o sucesso ou erro da operação.
    """
    try:
        # Criando a string de conexão
        conn_string = f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}?client_encoding=utf8'
        engine = create_engine(conn_string)

        # Conectar ao banco para exclusão, se necessário
        if excluir_antes:
            with psycopg2.connect(dbname=banco, user=usuario, password=senha, host=host, port=porta) as conn:
                with conn.cursor() as cursor:
                    cursor.execute(f"DELETE FROM {tabela};")
                    conn.commit()
                    print(f"🗑️ Todos os registros da tabela '{tabela}' foram excluídos.")

        # Substituir valores NaT e NaN por None (NULL no PostgreSQL)
        df = df.where(pd.notna(df), None)

        # Garantir que apenas colunas textuais sejam convertidas para string corretamente
        for col in df.select_dtypes(include=["object"]).columns:
            df[col] = df[col].apply(lambda x: str(x) if x is not None else None)

        # Contador para acompanhar a inserção por lotes
        total_registros = len(df)
        print(f"📊 Total de registros a serem inseridos: {total_registros}")

        for i in range(0, total_registros, batch_size):
            df_lote = df.iloc[i:i+batch_size]  # Pegando um lote de 1000 registros
            df_lote.to_sql(tabela, engine, if_exists='append', index=False, method='multi')
            print(f"✅ Inseridos {min(i+batch_size, total_registros)}/{total_registros} registros...")

        return "✅ Registros inseridos com sucesso!"

    except psycopg2.Error as e:
        return f"❌ Erro ao conectar ou executar query no PostgreSQL: {e}"
    
    except Exception as e:
        return f"❌ Erro ao inserir dados: {e}"


In [10]:
def ler_tabela_postgresql(tabela, host, porta, banco, usuario, senha):
    """
    Lê uma tabela do PostgreSQL e retorna como um DataFrame Pandas.

    Parâmetros:
    - tabela (str): Nome da tabela a ser lida.
    - host (str): Endereço do servidor PostgreSQL (exemplo: "localhost").
    - porta (str): Porta do PostgreSQL (exemplo: "5432").
    - banco (str): Nome do banco de dados.
    - usuario (str): Usuário do banco de dados.
    - senha (str): Senha do banco de dados.

    Retorna:
    - pd.DataFrame: DataFrame Pandas contendo os dados da tabela.
    """
    try:
        # Criando a string de conexão com PostgreSQL
        conn_string = f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}'
        engine = create_engine(conn_string)

        # Lendo a tabela para um DataFrame
        query = f"SELECT * FROM {tabela}"
        df = pd.read_sql(query, engine)

        print(f"✅ Tabela '{tabela}' carregada com sucesso! {df.shape[0]} registros encontrados.")
        return df

    except Exception as e:
        print(f"❌ Erro ao ler a tabela '{tabela}': {e}")
        return None


## 3) ETL

### 3.1) ETL ÓRGÃO

In [11]:
# Chamando a função
# url = "https://compras.dados.gov.br/licitacoes/v1/orgaos.json"
# chave = "Orgaos"

arquivo = "orgaos.parquet"

# df_orgaos = obter_dados_json(url, chave)

# df_orgaos.to_parquet(f"{diretorio}/{arquivo}", engine="pyarrow", index=False)

# Execucoes posteriores, utiliza o arquivo salvo
df_orgaos = pd.read_parquet(f"{diretorio}/{arquivo}", engine="fastparquet")

# Exibindo as primeiras linhas
df_orgaos.head(100)

,codigo,nome,codigo_tipo_adm,codigo_tipo_esfera,codigo_tipo_poder,ativo,_links.self.href,_links.self.title,_links.api_orcamento.href,_links.api_orcamento.title,_links.api_siconv.href,_links.api_siconv.title,_links.ocorrencia_fornecedores.href,_links.ocorrencia_fornecedores.title,_links.uasgs.href,_links.uasgs.title
0,1000,CAMARA DOS DEPUTADOS,1,Federal,1,True,/licitacoes/id/orgao/1000,Órgão 1000: CAMARA DOS DEPUTADOS,http://orcamento.dados.gov.br/doc/2014/Orgao/1000,Este órgão orçamentário sob o ponto de vista d...,http://api.convenios.gov.br/siconv/dados/orgao...,Este órgão orçamentário sob o ponto de vista d...,/fornecedores/v1/ocorrencias_fornecedores?id_o...,Ocorrências aplicadas por este órgão,/licitacoes/v1/uasgs?id_orgao=1000,UASGs associadas a este Órgão
1,1901,FUNDO ROTATIVO DA CAMARA DOS DEPUTADOS,7,None,1,True,/licitacoes/id/orgao/1901,Órgão 1901: FUNDO ROTATIVO DA CAMARA DOS DEPUT...,http://orcamento.dados.gov.br/doc/2014/Orgao/1901,Este órgão orçamentário sob o ponto de vista d...,http://api.convenios.gov.br/siconv/dados/orgao...,Este órgão orçamentário sob o ponto de vista d...,/fornecedores/v1/ocorrencias_fornecedores?id_o...,Ocorrências aplicadas por este órgão,/licitacoes/v1/uasgs?id_orgao=1901,UASGs associadas a este Órgão
2,2000,SENADO FEDERAL,1,None,1,True,/licitacoes/id/orgao/2000,Órgão 2000: SENADO FEDERAL,http://orcamento.dados.gov.br/doc/2014/Orgao/2000,Este órgão orçamentário sob o ponto de vista d...,http://api.convenios.gov.br/siconv/dados/orgao...,Este órgão orçamentário sob o ponto de vista d...,/fornecedores/v1/ocorrencias_fornecedores?id_o...,Ocorrências aplicadas por este órgão,/licitacoes/v1/uasgs?id_orgao=2000,UASGs associadas a este Órgão
3,2001,FUNDO ESPECIAL DO SENADO FEDERAL,7,None,1,True,/licitacoes/id/orgao/2001,Órgão 2001: FUNDO ESPECIAL DO SENADO FEDERAL,http://orcamento.dados.gov.br/doc/2014/Orgao/2001,Este órgão orçamentário sob o ponto de vista d...,http://api.convenios.gov.br/siconv/dados/orgao...,Este órgão orçamentário sob o ponto de vista d...,/fornecedores/v1/ocorrencias_fornecedores?id_o...,Ocorrências aplicadas por este órgão,/licitacoes/v1/uasgs?id_orgao=2001,UASGs associadas a este Órgão
4,2002,SENADO FEDERAL/FUNSEEP,7,None,1,True,/licitacoes/id/orgao/2002,Órgão 2002: SENADO FEDERAL/FUNSEEP,http://orcamento.dados.gov.br/doc/2014/Orgao/2002,Este órgão orçamentário sob o ponto de vista d...,http://api.convenios.gov.br/siconv/dados/orgao...,Este órgão orçamentário sob o ponto de vista d...,/fornecedores/v1/ocorrencias_fornecedores?id_o...,Ocorrências aplicadas por este órgão,/licitacoes/v1/uasgs?id_orgao=2002,UASGs associadas a este Órgão
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,22500,CIA.DE ENTREPOSTOS E ARMAZENS GER.DE S.PAULO,2,Federal,0,True,/licitacoes/id/orgao/22500,Órgão 22500: CIA.DE ENTREPOSTOS E ARMAZENS GER...,http://orcamento.dados.gov.br/doc/2014/Orgao/2...,Este órgão orçamentário sob o ponto de vista d...,http://api.convenios.gov.br/siconv/dados/orgao...,Este órgão orçamentário sob o ponto de vista d...,/fornecedores/v1/ocorrencias_fornecedores?id_o...,Ocorrências aplicadas por este órgão,/licitacoes/v1/uasgs?id_orgao=22500,UASGs associadas a este Órgão
96,22801,COMIS.EXEC.DO PLANO DA LAVOURA CACAUEIRA/MAPA,1,None,0,True,/licitacoes/id/orgao/22801,Órgão 22801: COMIS.EXEC.DO PLANO DA LAVOURA CA...,http://orcamento.dados.gov.br/doc/2014/Orgao/2...,Este órgão orçamentário sob o ponto de vista d...,http://api.convenios.gov.br/siconv/dados/orgao...,Este órgão orçamentário sob o ponto de vista d...,/fornecedores/v1/ocorrencias_fornecedores?id_o...,Ocorrências aplicadas por este órgão,/licitacoes/v1/uasgs?id_orgao=22801,UASGs associadas a este Órgão
97,22802,INSTITUTO NACIONAL DE METEOROLOGIA/MAPA,1,None,0,True,/licitacoes/id/orgao/22802,Órgão 22802: INSTITUTO NACIONAL DE METEOROLOGI...,http://orcamento.dados.gov.br/doc/2014/Orgao/2...,Este órgão orçamentário sob o ponto de vista d...,http://api.convenios.gov.br/siconv/dados/orgao...,Este ó

In [12]:
df_orgaos.shape

(9095, 16)

In [13]:
df_orgaos_tipo_esfera = df_orgaos.drop_duplicates(subset=['codigo_tipo_esfera'])[['codigo_tipo_esfera']].reset_index(drop=True)
df_orgaos_tipo_esfera.head(100)

,codigo_tipo_esfera
0,Federal
1,None
2,Estadual
3,Municipal


In [14]:
df_orgaos_tipo_esfera = df_orgaos.drop_duplicates(subset=['ativo'])[['ativo']].reset_index(drop=True)
df_orgaos_tipo_esfera.head(100)

,ativo
0,True
1,False


In [15]:
# Selecionar colunas específicas
df_orgaos_carga = df_orgaos[['codigo', 'nome', 'codigo_tipo_adm', 'codigo_tipo_esfera', 'codigo_tipo_poder', 'ativo']].copy()

df_orgaos_carga["ativo"] = df_orgaos_carga["ativo"].astype(int)

# Converter coluna "ativo" corretamente para boolean (True/False) antes da inserção
if "ativo" in df_orgaos_carga.columns:
    df_orgaos_carga["ativo"] = df_orgaos_carga["ativo"].apply(lambda x: True if x == 1 else False if x == 0 else None)

# Renomear colunas
df_orgaos_carga = df_orgaos_carga.rename(columns={
    "codigo_tipo_adm": "tipo_adm",
    "codigo_tipo_esfera": "tipo_esfera",
    "codigo_tipo_poder": "tipo_poder"
})

# Exibir as primeiras linhas
df_orgaos_carga.head()


,codigo,nome,tipo_adm,tipo_esfera,tipo_poder,ativo
0,1000,CAMARA DOS DEPUTADOS,1,Federal,1,True
1,1901,FUNDO ROTATIVO DA CAMARA DOS DEPUTADOS,7,None,1,True
2,2000,SENADO FEDERAL,1,None,1,True
3,2001,FUNDO ESPECIAL DO SENADO FEDERAL,7,None,1,True
4,2002,SENADO FEDERAL/FUNSEEP,7,None,1,True


In [16]:
df_tamanhos = verificar_tamanho_colunas(df_orgaos_carga)

print(df_tamanhos)

        Coluna  Tamanho Máximo
0       codigo               5
1         nome              45
2     tipo_adm               2
3  tipo_esfera               9
4   tipo_poder               1
5        ativo               5


In [17]:
# Insere dados de modalidade
# Chamando a função para inserir os dados
resultado = inserir_dados_postgresql(df_orgaos_carga, "orgao", DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASS, excluir_antes=True)

# Exibindo o resultado
print(resultado)

🗑️ Todos os registros da tabela 'orgao' foram excluídos.
📊 Total de registros a serem inseridos: 9095
✅ Inseridos 9095/9095 registros...
✅ Registros inseridos com sucesso!


### 3.2) ETL Modalidade

In [18]:
# Chamando a função
# url = "https://compras.dados.gov.br/licitacoes/v1/modalidades_licitacao.json"
# chave = "ModalidadesLicitacao"

arquivo = "modalidade.parquet"

# df_modalidades = obter_dados_json(url, chave)

# df_modalidades.to_parquet(f"{diretorio}/{arquivo}", engine="pyarrow", index=False)

# Execucoes posteriores, utiliza o arquivo salvo
df_modalidades = pd.read_parquet(f"{diretorio}/{arquivo}", engine="fastparquet")

# Exibindo as primeiras linhas
df_modalidades.head(100)

,codigo,descricao,_links.self.href,_links.self.title,_links.contratos.href,_links.contratos.title,_links.irps.href,_links.irps.title,_links.licitacoes.href,_links.licitacoes.title
0,1,CONVITE ...,/licitacoes/id/modalidade_licitacao/1,Modalidade da Licitação 1: CONVITE ...,/contratos/v1/contratos?modalidade=1,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=1,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=1,Licitações com esta modalidade
1,2,TOMADA DE PREÇOS ...,/licitacoes/id/modalidade_licitacao/2,Modalidade da Licitação 2: TOMADA DE PREÇOS ...,/contratos/v1/contratos?modalidade=2,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=2,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=2,Licitações com esta modalidade
2,3,CONCORRÊNCIA ...,/licitacoes/id/modalidade_licitacao/3,Modalidade da Licitação 3: CONCORRÊNCIA ...,/contratos/v1/contratos?modalidade=3,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=3,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=3,Licitações com esta modalidade
3,4,CONCORRÊNCIA INTERNACIONAL ...,/licitacoes/id/modalidade_licitacao/4,Modalidade da Licitação 4: CONCORRÊNCIA INTERN...,/contratos/v1/contratos?modalidade=4,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=4,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=4,Licitações com esta modalidade
4,5,PREGÃO ...,/licitacoes/id/modalidade_licitacao/5,Modalidade da Licitação 5: PREGÃO ...,/contratos/v1/contratos?modalidade=5,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=5,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=5,Licitações com esta modalidade
5,6,DISPENSA DE LICITAÇÃO ...,/licitacoes/id/modalidade_licitacao/6,Modalidade da Licitação 6: DISPENSA DE LICITAÇ...,/contratos/v1/contratos?modalidade=6,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=6,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=6,Licitações com esta modalidade
6,7,INEXIGIBILIDADE DE LICITAÇÃO ...,/licitacoes/id/modalidade_licitacao/7,Modalidade da Licitação 7: INEXIGIBILIDADE DE ...,/contratos/v1/contratos?modalidade=7,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=7,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=7,Licitações com esta modalidade
7,20,CONCURSO ...,/licitacoes/id/modalidade_licitacao/20,Modalidade da Licitação 20: CONCURSO ...,/contratos/v1/contratos?modalidade=20,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=20,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=20,Licitações com esta modalidade
8,22,TOMADA DE PREÇOS POR TÉCNICA E PREÇO ...,/licitacoes/id/modalidade_licitacao/22,Modalidade da Licitação 22: TOMADA DE PREÇOS P...,/contratos/v1/contratos?modalidade=22,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=22,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=22,Licitações com esta modalidade
9,33,CONCORRÊNCIA POR TÉCNICA E PREÇO ...,/licitacoes/id/modalidade_licitacao/33,Modalidade da Licitação 33: CONCORRÊNCIA POR T...,/contratos/v1/contratos?modalidade=33,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=33,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=33,Licitações com esta modalidade


In [19]:
df_modalidades.shape

(11, 10)

In [20]:
# Incluir no dataframe de modalidade a 9999 e 9997
nova_linha = {'codigo': 9999, 'descricao': 'Pregão - Registro de Preço'.upper(), '_links': None}
df_modalidades = pd.concat([df_modalidades, pd.DataFrame([nova_linha])], ignore_index=True)

nova_linha = {'codigo': 9997, 'descricao': 'Concorrência - Registro de Preço'.upper(), '_links': None}
df_modalidades = pd.concat([df_modalidades, pd.DataFrame([nova_linha])], ignore_index=True)

# Exibindo as linhas para verificar
df_modalidades.head(100)

,codigo,descricao,_links.self.href,_links.self.title,_links.contratos.href,_links.contratos.title,_links.irps.href,_links.irps.title,_links.licitacoes.href,_links.licitacoes.title,_links
0,1,CONVITE ...,/licitacoes/id/modalidade_licitacao/1,Modalidade da Licitação 1: CONVITE ...,/contratos/v1/contratos?modalidade=1,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=1,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=1,Licitações com esta modalidade,NaN
1,2,TOMADA DE PREÇOS ...,/licitacoes/id/modalidade_licitacao/2,Modalidade da Licitação 2: TOMADA DE PREÇOS ...,/contratos/v1/contratos?modalidade=2,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=2,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=2,Licitações com esta modalidade,NaN
2,3,CONCORRÊNCIA ...,/licitacoes/id/modalidade_licitacao/3,Modalidade da Licitação 3: CONCORRÊNCIA ...,/contratos/v1/contratos?modalidade=3,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=3,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=3,Licitações com esta modalidade,NaN
3,4,CONCORRÊNCIA INTERNACIONAL ...,/licitacoes/id/modalidade_licitacao/4,Modalidade da Licitação 4: CONCORRÊNCIA INTERN...,/contratos/v1/contratos?modalidade=4,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=4,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=4,Licitações com esta modalidade,NaN
4,5,PREGÃO ...,/licitacoes/id/modalidade_licitacao/5,Modalidade da Licitação 5: PREGÃO ...,/contratos/v1/contratos?modalidade=5,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=5,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=5,Licitações com esta modalidade,NaN
5,6,DISPENSA DE LICITAÇÃO ...,/licitacoes/id/modalidade_licitacao/6,Modalidade da Licitação 6: DISPENSA DE LICITAÇ...,/contratos/v1/contratos?modalidade=6,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=6,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=6,Licitações com esta modalidade,NaN
6,7,INEXIGIBILIDADE DE LICITAÇÃO ...,/licitacoes/id/modalidade_licitacao/7,Modalidade da Licitação 7: INEXIGIBILIDADE DE ...,/contratos/v1/contratos?modalidade=7,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=7,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=7,Licitações com esta modalidade,NaN
7,20,CONCURSO ...,/licitacoes/id/modalidade_licitacao/20,Modalidade da Licitação 20: CONCURSO ...,/contratos/v1/contratos?modalidade=20,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=20,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=20,Licitações com esta modalidade,NaN
8,22,TOMADA DE PREÇOS POR TÉCNICA E PREÇO ...,/licitacoes/id/modalidade_licitacao/22,Modalidade da Licitação 22: TOMADA DE PREÇOS P...,/contratos/v1/contratos?modalidade=22,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=22,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=22,Licitações com esta modalidade,NaN
9,33,CONCORRÊNCIA POR TÉCNICA E PREÇO ...,/licitacoes/id/modalidade_licitacao/33,Modalidade da Licitação 33: CONCORRÊNCIA POR T...,/contratos/v1/contratos?modalidade=33,Contratos até 2020 que originaram de licitaçõe...,/licitacoes/v1/irps?modalidade_licitacao=33,IRPs que resultaram em licitações com esta mod...,/licitacoes/v1/licitacoes?modalidade=33,Licitações com esta modalidade,NaN


In [21]:
df_modalidades_carga = df_modalidades[['codigo', 'descricao' ]]
# df_modalidades_carga = df_modalidades_carga.astype(str).apply(lambda x: x.str.encode('utf-8', 'ignore').str.decode('utf-8'))
# df_modalidades_carga['descricao'] = df_modalidades_carga['descricao'].str.normalize('NFKD').str.encode('ascii', 'ignore').str.decode('utf-8')

df_modalidades_carga.head(100)

,codigo,descricao
0,1,CONVITE ...
1,2,TOMADA DE PREÇOS ...
2,3,CONCORRÊNCIA ...
3,4,CONCORRÊNCIA INTERNACIONAL ...
4,5,PREGÃO ...
5,6,DISPENSA DE LICITAÇÃO ...
6,7,INEXIGIBILIDADE DE LICITAÇÃO ...
7,20,CONCURSO ...
8,22,TOMADA DE PREÇOS POR TÉCNICA E PREÇO ...
9,33,CONCORRÊNCIA POR TÉCNICA E PREÇO ...


In [22]:
# Insere dados de modalidade
resultado = inserir_dados_postgresql(df_modalidades_carga, "modalidade", DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASS, excluir_antes=True)

# Exibindo o resultado
print(resultado)

🗑️ Todos os registros da tabela 'modalidade' foram excluídos.
📊 Total de registros a serem inseridos: 13
✅ Inseridos 13/13 registros...
✅ Registros inseridos com sucesso!


### 3.3) ETL UF

In [23]:
import pandas as pd

# Lista de estados do Brasil
estados = [
    ("AC", "Acre"), ("AL", "Alagoas"), ("AP", "Amapá"), ("AM", "Amazonas"),
    ("BA", "Bahia"), ("CE", "Ceará"), ("DF", "Distrito Federal"), ("ES", "Espírito Santo"),
    ("GO", "Goiás"), ("MA", "Maranhão"), ("MT", "Mato Grosso"), ("MS", "Mato Grosso do Sul"),
    ("MG", "Minas Gerais"), ("PA", "Pará"), ("PB", "Paraíba"), ("PR", "Paraná"),
    ("PE", "Pernambuco"), ("PI", "Piauí"), ("RJ", "Rio de Janeiro"), ("RN", "Rio Grande do Norte"),
    ("RS", "Rio Grande do Sul"), ("RO", "Rondônia"), ("RR", "Roraima"), ("SC", "Santa Catarina"),
    ("SP", "São Paulo"), ("SE", "Sergipe"), ("TO", "Tocantins"), ("-3", "Inválido")
]

# Criar o DataFrame
df_estados = pd.DataFrame(estados, columns=["sigla", "descricao"])

# Exibir o DataFrame
df_estados.head()


,sigla,descricao
0,AC,Acre
1,AL,Alagoas
2,AP,Amapá
3,AM,Amazonas
4,BA,Bahia


In [24]:
# Insere dados de modalidade
resultado = inserir_dados_postgresql(df_estados, "uf", DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASS, excluir_antes=True)

# Exibindo o resultado
print(resultado)

🗑️ Todos os registros da tabela 'uf' foram excluídos.
📊 Total de registros a serem inseridos: 28
✅ Inseridos 28/28 registros...
✅ Registros inseridos com sucesso!


### 3.4) ETL MUNICÍPIO

In [25]:
df_licitacoes = processar_diretorio(diretorio, "Licitação")
df_licitacoes.head()

✅ 3 arquivos processados e 5523 registros carregados.


,Número Licitação,Código UG,Nome UG,Código Modalidade Compra,Modalidade Compra,Número Processo,Objeto,Situação Licitação,Código Órgão Superior,Nome Órgão Superior,Código Órgão,Nome Órgão,UF,Município,Data Resultado Compra,Data Abertura,Valor Licitação,ano,mes
0,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,"Objeto: Pregão Eletrônico - Aquisição, de 8 (o...",Publicado,52000,Ministério da Defesa,52000,Ministério da Defesa - Unidades com vínculo,DF,BRASILIA,15/02/2024,06/02/2024,"228240,0000",2024,02
1,12023,120628,GRUPAMENTO DE APOIO DE BELEM,2,Tomada de Preços,67215006797202317,Objeto: Contratação de empresa de engenharia e...,Publicado,52000,Ministério da Defesa,52111,Comando da Aeronáutica,PA,BELEM,07/02/2024,NaN,"955059,8100",2024,02
2,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,Objeto: Pregão Eletrônico - Contratação dos se...,Evento de Suspensão Publicado,26000,Ministério da Educação,26435,Instituto Federal do Rio Grande do Norte,RN,NATAL,29/02/2024,24/01/2024,"752203,3200",2024,02
3,12023,152803,INST.FED.DE ALAGOAS/CAMPUS MURICI,9999,Pregão - Registro de Preço,23041008283202329,Objeto: Pregão Eletrônico - Aquisição de ferra...,Evento de Resultado de Julgame,26000,Ministério da Educação,26402,Instituto Federal de Alagoas,AL,MURICI,16/02/2024,10/01/2024,"855,2800",2024,02
4,12023,153015,CENTRO FEDERAL DE EDUCACAO TECNOLOGICA DE MG,3,Concorrência,23062020648202378,Objeto: Contratação de empresa especializada p...,Publicado,26000,Ministério da Educação,26257,Centro Federal de Educação Tecnológica de,MG,BELO HORIZONTE,16/02/2024,NaN,"4500000,0000",2024,02


In [26]:
df_licitacoes.columns

Index(['Número Licitação', 'Código UG', 'Nome UG', 'Código Modalidade Compra',
       'Modalidade Compra', 'Número Processo', 'Objeto', 'Situação Licitação',
       'Código Órgão Superior', 'Nome Órgão Superior', 'Código Órgão',
       'Nome Órgão', 'UF', 'Município', 'Data Resultado Compra',
       'Data Abertura', 'Valor Licitação', 'ano', 'mes'],
      dtype='object')

In [27]:
df_licitacoes = df_licitacoes.rename(columns={'Número Licitação': 'numero', 
                                              'Código UG': 'codigo_ug',
                                              'Nome UG': 'nome_ug',
                                              'Código Modalidade Compra': 'modalidade_codigo',
                                              'Modalidade Compra': 'modalidade_desc',
                                              'Número Processo': 'num_processo',
                                              'Objeto':'objeto',
                                              'Situação Licitação': 'situacao',
                                              'Código Órgão Superior': 'orgao_codigo_sup',
                                              'Nome Órgão Superior': 'orgao_nome_sup',
                                              'Código Órgão': 'orgao_codigo',
                                              'Nome Órgão': 'orgao_nome', 
                                              'UF': 'uf',
                                              'Município': 'municipio',
                                              'Data Resultado Compra': 'data_compra',
                                              'Data Abertura': 'data_abertura',
                                              'Valor Licitação': 'valor'})


# Criar uma nova coluna concatenada
df_licitacoes["chave_unica"] = (
    df_licitacoes["numero"].astype(str) + "_" +
    df_licitacoes["codigo_ug"].astype(str) + "_" +
    df_licitacoes["modalidade_codigo"].astype(str) + "_" +
    df_licitacoes["ano"].astype(str) + "_" +
    df_licitacoes["mes"].astype(str).str.zfill(2)
)

df_licitacoes.head()

,numero,codigo_ug,nome_ug,modalidade_codigo,modalidade_desc,num_processo,objeto,situacao,orgao_codigo_sup,orgao_nome_sup,orgao_codigo,orgao_nome,uf,municipio,data_compra,data_abertura,valor,ano,mes,chave_unica
0,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,"Objeto: Pregão Eletrônico - Aquisição, de 8 (o...",Publicado,52000,Ministério da Defesa,52000,Ministério da Defesa - Unidades com vínculo,DF,BRASILIA,15/02/2024,06/02/2024,"228240,0000",2024,02,12023_110511_5_2024_02
1,12023,120628,GRUPAMENTO DE APOIO DE BELEM,2,Tomada de Preços,67215006797202317,Objeto: Contratação de empresa de engenharia e...,Publicado,52000,Ministério da Defesa,52111,Comando da Aeronáutica,PA,BELEM,07/02/2024,NaN,"955059,8100",2024,02,12023_120628_2_2024_02
2,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,Objeto: Pregão Eletrônico - Contratação dos se...,Evento de Suspensão Publicado,26000,Ministério da Educação,26435,Instituto Federal do Rio Grande do Norte,RN,NATAL,29/02/2024,24/01/2024,"752203,3200",2024,02,12023_152711_5_2024_02
3,12023,152803,INST.FED.DE ALAGOAS/CAMPUS MURICI,9999,Pregão - Registro de Preço,23041008283202329,Objeto: Pregão Eletrônico - Aquisição de ferra...,Evento de Resultado de Julgame,26000,Ministério da Educação,26402,Instituto Federal de Alagoas,AL,MURICI,16/02/2024,10/01/2024,"855,2800",2024,02,12023_152803_9999_2024_02
4,12023,153015,CENTRO FEDERAL DE EDUCACAO TECNOLOGICA DE MG,3,Concorrência,23062020648202378,Objeto: Contratação de empresa especializada p...,Publicado,26000,Ministério da Educação,26257,Centro Federal de Educação Tecnológica de,MG,BELO HORIZONTE,16/02/2024,NaN,"4500000,0000",2024,02,12023_153015_3_2024_02


In [28]:
df_municipios = df_licitacoes[['uf', 'municipio']].copy()
df_municipios.rename(columns={"uf": "uf_sigla", "municipio": "descricao"}, inplace=True)

df_municipios.head()


,uf_sigla,descricao
0,DF,BRASILIA
1,PA,BELEM
2,RN,NATAL
3,AL,MURICI
4,MG,BELO HORIZONTE


In [29]:
# Insere dados
resultado = inserir_dados_postgresql(df_municipios, "municipio", DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASS, excluir_antes=True)

# Exibindo o resultado
print(resultado)

🗑️ Todos os registros da tabela 'municipio' foram excluídos.
📊 Total de registros a serem inseridos: 5523
✅ Inseridos 5523/5523 registros...
✅ Registros inseridos com sucesso!


In [30]:
# Chamar a função para carregar a tabela do PostgreSQL
df_municipios_postgree = ler_tabela_postgresql("municipio", DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASS)

# Exibir as 5 primeiras linhas
df_municipios_postgree.head()

✅ Tabela 'municipio' carregada com sucesso! 5523 registros encontrados.


,id,uf_sigla,descricao
0,1,DF,BRASILIA
1,2,PA,BELEM
2,3,RN,NATAL
3,4,AL,MURICI
4,5,MG,BELO HORIZONTE


### 3.5) ETL LICITACAO

In [31]:
# Fazer o merge
df_licitacoes = df_licitacoes.merge(
    df_municipios_postgree[["id", "uf_sigla", "descricao"]],  # Apenas colunas necessárias
    left_on=["uf", "municipio"],  # Colunas para correspondência
    right_on=["uf_sigla", "descricao"],
    how="left"  # Mantém todas as linhas de df_licitacoes_carga
)

# Renomear a coluna 'id' para 'municipio_id'
df_licitacoes.rename(columns={"id": "municipio_id"}, inplace=True)

# Exibir os primeiros registros
df_licitacoes.head(5)

,numero,codigo_ug,nome_ug,modalidade_codigo,modalidade_desc,num_processo,objeto,situacao,orgao_codigo_sup,orgao_nome_sup,...,municipio,data_compra,data_abertura,valor,ano,mes,chave_unica,municipio_id,uf_sigla,descricao
0,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,"Objeto: Pregão Eletrônico - Aquisição, de 8 (o...",Publicado,52000,Ministério da Defesa,...,BRASILIA,15/02/2024,06/02/2024,"228240,0000",2024,02,12023_110511_5_2024_02,1,DF,BRASILIA
1,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,"Objeto: Pregão Eletrônico - Aquisição, de 8 (o...",Publicado,52000,Ministério da Defesa,...,BRASILIA,15/02/2024,06/02/2024,"228240,0000",2024,02,12023_110511_5_2024_02,18,DF,BRASILIA
2,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,"Objeto: Pregão Eletrônico - Aquisição, de 8 (o...",Publicado,52000,Ministério da Defesa,...,BRASILIA,15/02/2024,06/02/2024,"228240,0000",2024,02,12023_110511_5_2024_02,32,DF,BRASILIA
3,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,"Objeto: Pregão Eletrônico - Aquisição, de 8 (o...",Publicado,52000,Ministério da Defesa,...,BRASILIA,15/02/2024,06/02/2024,"228240,0000",2024,02,12023_110511_5_2024_02,45,DF,BRASILIA
4,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,"Objeto: Pregão Eletrônico - Aquisição, de 8 (o...",Publicado,52000,Ministério da Defesa,...,BRASILIA,15/02/2024,06/02/2024,"228240,0000",2024,02,12023_110511_5_2024_02,59,DF,BRASILIA


In [32]:
# Criar uma cópia do DataFrame apenas com as colunas relevantes
df_licitacoes_carga = df_licitacoes[
    [
        "numero",
        "codigo_ug",
        "modalidade_codigo",
        "num_processo",
        "objeto",
        "situacao",
        "orgao_codigo",
        "municipio_id",
        "data_compra",
        "data_abertura",
        "valor",
        "ano",
        "mes",
        "chave_unica"
    ]
].copy()

# Substituir valores inválidos ('None', 'nan', np.nan, '') por None (NULL)
df_licitacoes_carga.replace({"None": None, "nan": None, np.nan: None, "": None}, inplace=True)

# Converter colunas de data para o formato correto ('YYYY-MM-DD') SEM '00:00:00'
df_licitacoes_carga["data_compra"] = pd.to_datetime(
    df_licitacoes_carga["data_compra"], format="%d/%m/%Y", errors="coerce"
).dt.date

df_licitacoes_carga["data_abertura"] = pd.to_datetime(
    df_licitacoes_carga["data_abertura"], format="%d/%m/%Y", errors="coerce"
).dt.date

# Garantir que os valores numéricos estejam no formato correto
df_licitacoes_carga["valor"] = (
    df_licitacoes_carga["valor"]
    .astype(str)
    .str.replace(",", ".", regex=True)
    .astype(float)
)

# Converter NaT para None antes da inserção no PostgreSQL
# Converter colunas de data corretamente para evitar erro no PostgreSQL
df_licitacoes_carga["data_compra"] = df_licitacoes_carga["data_compra"].apply(lambda x: x if pd.notna(x) else None)
df_licitacoes_carga["data_abertura"] = df_licitacoes_carga["data_abertura"].apply(lambda x: x if pd.notna(x) else None)

# Garantir que colunas de data sejam tratadas corretamente e passem None (NULL) no PostgreSQL
df_licitacoes_carga["data_compra"] = df_licitacoes_carga["data_compra"].apply(lambda x: None if pd.isna(x) else x)
df_licitacoes_carga["data_abertura"] = df_licitacoes_carga["data_abertura"].apply(lambda x: None if pd.isna(x) else x)


# Conferir se os valores nulos foram corretamente ajustados
print(df_licitacoes_carga[["data_compra", "data_abertura", "valor"]].head(10))

  data_compra data_abertura     valor
0  2024-02-15    2024-02-06  228240.0
1  2024-02-15    2024-02-06  228240.0
2  2024-02-15    2024-02-06  228240.0
3  2024-02-15    2024-02-06  228240.0
4  2024-02-15    2024-02-06  228240.0
5  2024-02-15    2024-02-06  228240.0
6  2024-02-15    2024-02-06  228240.0
7  2024-02-15    2024-02-06  228240.0
8  2024-02-15    2024-02-06  228240.0
9  2024-02-15    2024-02-06  228240.0


In [33]:
# Insere dados de licitacao
resultado = inserir_dados_postgresql(df_licitacoes_carga, "licitacao", DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASS, excluir_antes=True)

# Exibindo o resultado
print(resultado)

🗑️ Todos os registros da tabela 'licitacao' foram excluídos.
📊 Total de registros a serem inseridos: 1273555
✅ Inseridos 10000/1273555 registros...
✅ Inseridos 20000/1273555 registros...
✅ Inseridos 30000/1273555 registros...
✅ Inseridos 40000/1273555 registros...
✅ Inseridos 50000/1273555 registros...
✅ Inseridos 60000/1273555 registros...
✅ Inseridos 70000/1273555 registros...
✅ Inseridos 80000/1273555 registros...
✅ Inseridos 90000/1273555 registros...
✅ Inseridos 100000/1273555 registros...
✅ Inseridos 110000/1273555 registros...
✅ Inseridos 120000/1273555 registros...
✅ Inseridos 130000/1273555 registros...
✅ Inseridos 140000/1273555 registros...
✅ Inseridos 150000/1273555 registros...
✅ Inseridos 160000/1273555 registros...
✅ Inseridos 170000/1273555 registros...
✅ Inseridos 180000/1273555 registros...
✅ Inseridos 190000/1273555 registros...
✅ Inseridos 200000/1273555 registros...
✅ Inseridos 210000/1273555 registros...
✅ Inseridos 220000/1273555 registros...
✅ Inseridos 230000/1

In [34]:
# Chamar a função para carregar a tabela do PostgreSQL
df_licitacoes_postgree = ler_tabela_postgresql("licitacao", DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASS)

# Exibir as 5 primeiras linhas
df_licitacoes_postgree.head()

✅ Tabela 'licitacao' carregada com sucesso! 1273555 registros encontrados.


,id,numero,orgao_codigo,codigo_ug,modalidade_codigo,ano,mes,chave_unica,num_processo,objeto,situacao,municipio_id,data_compra,data_abertura,valor
0,1,12023,52000,110511,5,2024,2,12023_110511_5_2024_02,60090000453202210,"Objeto: Pregão Eletrônico - Aquisição, de 8 (o...",Publicado,1,2024-02-15,2024-02-06,228240.0
1,2,12023,52000,110511,5,2024,2,12023_110511_5_2024_02,60090000453202210,"Objeto: Pregão Eletrônico - Aquisição, de 8 (o...",Publicado,18,2024-02-15,2024-02-06,228240.0
2,3,12023,52000,110511,5,2024,2,12023_110511_5_2024_02,60090000453202210,"Objeto: Pregão Eletrônico - Aquisição, de 8 (o...",Publicado,32,2024-02-15,2024-02-06,228240.0
3,4,12023,52000,110511,5,2024,2,12023_110511_5_2024_02,60090000453202210,"Objeto: Pregão Eletrônico - Aquisição, de 8 (o...",Publicado,45,2024-02-15,2024-02-06,228240.0
4,5,12023,52000,110511,5,2024,2,12023_110511_5_2024_02,60090000453202210,"Objeto: Pregão Eletrônico - Aquisição, de 8 (o...",Publicado,59,2024-02-15,2024-02-06,228240.0


### 3.5) ETL CNPJ

In [35]:
df_participantes = processar_diretorio(diretorio, "ParticipantesLicitação")
df_participantes.head()

✅ 3 arquivos processados e 347416 registros carregados.


,Número Licitação,Código UG,Nome UG,Código Modalidade Compra,Modalidade Compra,Número Processo,Código Órgão,Nome Órgão,Código Item Compra,Descrição Item Compra,Código Participante,Nome Participante,Flag Vencedor,ano,mes
0,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,46955586000149,M5 DRONES LTDA,NÃO,2024,02
1,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,30001450000160,30.001.450 CLEVERSON ANTONIO MACHADO SOARES,NÃO,2024,02
2,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,50915513000109,ALBACH E CHAVES ASSOCIADOS LTDA,NÃO,2024,02
3,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,52918773000155,DASP COMERCIO E SERVICOS TECNOLOGICOS LTDA,NÃO,2024,02
4,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,12007998000135,PISONTEC COMERCIO E SERVICOS EM TECNOLOGIA DA ...,NÃO,2024,02


In [36]:
df_participantes.columns

Index(['Número Licitação', 'Código UG', 'Nome UG', 'Código Modalidade Compra',
       'Modalidade Compra', 'Número Processo', 'Código Órgão', 'Nome Órgão',
       'Código Item Compra', 'Descrição Item Compra', 'Código Participante',
       'Nome Participante', 'Flag Vencedor', 'ano', 'mes'],
      dtype='object')

In [37]:
df_participantes = df_participantes.rename(columns={'Número Licitação': 'numero', 
                                              'Código UG': 'codigo_ug',
                                              'Nome UG': 'nome_ug',
                                              'Código Modalidade Compra': 'modalidade_codigo',
                                              'Modalidade Compra': 'modalidade_desc',
                                              'Número Processo': 'num_processo',
                                              'Código Órgão': 'orgao_codigo',
                                              'Nome Órgão': 'orgao_nome', 
                                              'Código Item Compra': 'item_codigo',
                                              'Descrição Item Compra': 'item_descricao',
                                              'Código Participante': 'cnpj_codigo',
                                              'Nome Participante': 'cnpj_nome',
                                              'Flag Vencedor': 'vencedor_flag'})

df_participantes.head()

,numero,codigo_ug,nome_ug,modalidade_codigo,modalidade_desc,num_processo,orgao_codigo,orgao_nome,item_codigo,item_descricao,cnpj_codigo,cnpj_nome,vencedor_flag,ano,mes
0,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,46955586000149,M5 DRONES LTDA,NÃO,2024,02
1,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,30001450000160,30.001.450 CLEVERSON ANTONIO MACHADO SOARES,NÃO,2024,02
2,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,50915513000109,ALBACH E CHAVES ASSOCIADOS LTDA,NÃO,2024,02
3,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,52918773000155,DASP COMERCIO E SERVICOS TECNOLOGICOS LTDA,NÃO,2024,02
4,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,12007998000135,PISONTEC COMERCIO E SERVICOS EM TECNOLOGIA DA ...,NÃO,2024,02


In [38]:
df_itens_licitacao = processar_diretorio(diretorio, "ItemLicitação")
df_itens_licitacao.head()

✅ 3 arquivos processados e 123873 registros carregados.


,Número Licitação,Código UG,Nome UG,Código Modalidade Compra,Modalidade Compra,Número Processo,Código Órgão,Nome Órgão,Código Item Compra,Descrição,Quantidade Item,Valor Item,Código Vencedor,Nome Vencedor,ano,mes
0,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,8,"228240,0000",09240519000111,SOFTWARE.COM.BR TECNOLOGIA E CONSULTORIA LTDA.,2024,02
1,12023,120628,GRUPAMENTO DE APOIO DE BELEM,2,Tomada de Preços,67215006797202317,52111,Comando da Aeronáutica,1206280200001202300001,MANUTENCAO / REFORMA PREDIAL,1,"955059,8100",42780861000180,RAVENA INVEST LTDA,2024,02
2,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300001,MANUTENCAO / REFORMA PREDIAL,24,"166229,7600",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02
3,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300002,SERVICO ODONTOLOGICO,12,"37385,5200",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02
4,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300003,PRESTACAO DE SERVICOS DE ELETRICISTA,12,"88556,4000",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02


In [39]:
df_itens_licitacao.columns

Index(['Número Licitação', 'Código UG', 'Nome UG', 'Código Modalidade Compra',
       'Modalidade Compra', 'Número Processo', 'Código Órgão', 'Nome Órgão',
       'Código Item Compra', 'Descrição', 'Quantidade Item', 'Valor Item',
       'Código Vencedor', 'Nome Vencedor', 'ano', 'mes'],
      dtype='object')

In [40]:
df_itens_licitacao = df_itens_licitacao.rename(columns={'Número Licitação': 'numero', 
                                              'Código UG': 'codigo_ug',
                                              'Nome UG': 'nome_ug',
                                              'Código Modalidade Compra': 'modalidade_codigo',
                                              'Modalidade Compra': 'modalidade_desc',
                                              'Número Processo': 'num_processo',
                                              'Código Órgão': 'orgao_codigo',
                                              'Nome Órgão': 'orgao_nome', 
                                              'Código Item Compra': 'item_compra_codigo',
                                              'Descrição': 'item_compra_descricao',
                                              'Quantidade Item': 'item_compra_qtde',
                                              'Valor Item': 'item_compra_valor',
                                              'Código Vencedor': 'vencedor_codigo', 
                                              'Nome Vencedor': 'vencedor_nome'})


df_itens_licitacao.head()

,numero,codigo_ug,nome_ug,modalidade_codigo,modalidade_desc,num_processo,orgao_codigo,orgao_nome,item_compra_codigo,item_compra_descricao,item_compra_qtde,item_compra_valor,vencedor_codigo,vencedor_nome,ano,mes
0,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,8,"228240,0000",09240519000111,SOFTWARE.COM.BR TECNOLOGIA E CONSULTORIA LTDA.,2024,02
1,12023,120628,GRUPAMENTO DE APOIO DE BELEM,2,Tomada de Preços,67215006797202317,52111,Comando da Aeronáutica,1206280200001202300001,MANUTENCAO / REFORMA PREDIAL,1,"955059,8100",42780861000180,RAVENA INVEST LTDA,2024,02
2,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300001,MANUTENCAO / REFORMA PREDIAL,24,"166229,7600",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02
3,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300002,SERVICO ODONTOLOGICO,12,"37385,5200",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02
4,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300003,PRESTACAO DE SERVICOS DE ELETRICISTA,12,"88556,4000",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02


In [41]:
# Criar um novo DataFrame apenas com as colunas relevantes de cada DataFrame
df_participantes_selected = df_participantes[["cnpj_codigo", "cnpj_nome"]].copy()
df_itens_licitacao_selected = df_itens_licitacao[["vencedor_codigo", "vencedor_nome"]].copy()

# Concatenar os DataFrames, mantendo as colunas distintas
df_consolidado = pd.concat([
    df_participantes_selected.rename(columns={"cnpj_codigo": "codigo", "cnpj_nome": "nome"}),
    df_itens_licitacao_selected.rename(columns={"vencedor_codigo": "codigo", "vencedor_nome": "nome"})
], ignore_index=True)

# Remover duplicatas baseando-se nas colunas 'codigo' e 'nome'
df_cpf = df_consolidado.drop_duplicates(subset=["codigo", "nome"])

# Exibir o DataFrame final
df_cpf.head()


,codigo,nome
0,46955586000149,M5 DRONES LTDA
1,30001450000160,30.001.450 CLEVERSON ANTONIO MACHADO SOARES
2,50915513000109,ALBACH E CHAVES ASSOCIADOS LTDA
3,52918773000155,DASP COMERCIO E SERVICOS TECNOLOGICOS LTDA
4,12007998000135,PISONTEC COMERCIO E SERVICOS EM TECNOLOGIA DA ...


In [42]:
df_cpf.shape

(12084, 2)

In [43]:
df_duplicados = df_cpf[df_cpf.duplicated(subset=['codigo'], keep=False)]
df_duplicados

,codigo,nome


In [44]:
# Insere dados de cnpj
resultado = inserir_dados_postgresql(df_cpf, "cnpj", DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASS, excluir_antes=True)

# Exibindo o resultado
print(resultado)

🗑️ Todos os registros da tabela 'cnpj' foram excluídos.
📊 Total de registros a serem inseridos: 12084
✅ Inseridos 10000/12084 registros...
✅ Inseridos 12084/12084 registros...
✅ Registros inseridos com sucesso!


### 3.6) ETL ITEM

In [45]:
df_itens_licitacao = processar_diretorio(diretorio, "ItemLicitação")
df_itens_licitacao.head()

✅ 3 arquivos processados e 123873 registros carregados.


,Número Licitação,Código UG,Nome UG,Código Modalidade Compra,Modalidade Compra,Número Processo,Código Órgão,Nome Órgão,Código Item Compra,Descrição,Quantidade Item,Valor Item,Código Vencedor,Nome Vencedor,ano,mes
0,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,8,"228240,0000",09240519000111,SOFTWARE.COM.BR TECNOLOGIA E CONSULTORIA LTDA.,2024,02
1,12023,120628,GRUPAMENTO DE APOIO DE BELEM,2,Tomada de Preços,67215006797202317,52111,Comando da Aeronáutica,1206280200001202300001,MANUTENCAO / REFORMA PREDIAL,1,"955059,8100",42780861000180,RAVENA INVEST LTDA,2024,02
2,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300001,MANUTENCAO / REFORMA PREDIAL,24,"166229,7600",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02
3,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300002,SERVICO ODONTOLOGICO,12,"37385,5200",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02
4,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300003,PRESTACAO DE SERVICOS DE ELETRICISTA,12,"88556,4000",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02


In [46]:
df_itens_licitacao = df_itens_licitacao.rename(columns={'Número Licitação': 'numero', 
                                              'Código UG': 'codigo_ug',
                                              'Nome UG': 'nome_ug',
                                              'Código Modalidade Compra': 'modalidade_codigo',
                                              'Modalidade Compra': 'modalidade_desc',
                                              'Número Processo': 'num_processo',
                                              'Objeto':'objeto',
                                              'Situação Licitação': 'situacao',
                                              'Código Órgão Superior': 'orgao_codigo_sup',
                                              'Nome Órgão Superior': 'orgao_nome_sup',
                                              'Código Órgão': 'orgao_codigo',
                                              'Nome Órgão': 'orgao_nome', 
                                              'Código Item Compra': 'item_codigo',
                                              'Descrição': 'item_descricao',
                                              'Quantidade Item': 'item_qtde',
                                              'Valor Item': 'item_valor',
                                              'Código Vencedor': 'vencedor_cnpj',
                                              'Nome Vencedor': 'vencedor_nome',
                                              'UF': 'uf',
                                              'Município': 'municipio',
                                              'Data Resultado Compra': 'data_compra',
                                              'Data Abertura': 'data_abertura',
                                              'Valor Licitação': 'valor'})

df_itens_licitacao.head()

,numero,codigo_ug,nome_ug,modalidade_codigo,modalidade_desc,num_processo,orgao_codigo,orgao_nome,item_codigo,item_descricao,item_qtde,item_valor,vencedor_cnpj,vencedor_nome,ano,mes
0,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,8,"228240,0000",09240519000111,SOFTWARE.COM.BR TECNOLOGIA E CONSULTORIA LTDA.,2024,02
1,12023,120628,GRUPAMENTO DE APOIO DE BELEM,2,Tomada de Preços,67215006797202317,52111,Comando da Aeronáutica,1206280200001202300001,MANUTENCAO / REFORMA PREDIAL,1,"955059,8100",42780861000180,RAVENA INVEST LTDA,2024,02
2,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300001,MANUTENCAO / REFORMA PREDIAL,24,"166229,7600",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02
3,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300002,SERVICO ODONTOLOGICO,12,"37385,5200",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02
4,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300003,PRESTACAO DE SERVICOS DE ELETRICISTA,12,"88556,4000",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02


In [47]:
df_itens_licitacao['item_codigo'] = df_itens_licitacao.apply(lambda row: (
                        f"{str(row['codigo_ug']).zfill(6)}"
                        f"{str(row['modalidade_codigo']).zfill(2)}"
                        f"{str(row['numero']).zfill(5)}"
                        f"{str(row['ano']).zfill(4)}"
                        f"99999"
                    ) if pd.isnull(row['item_codigo']) else row['item_codigo'], axis=1)

In [48]:
df_itens_licitacao.head()

,numero,codigo_ug,nome_ug,modalidade_codigo,modalidade_desc,num_processo,orgao_codigo,orgao_nome,item_codigo,item_descricao,item_qtde,item_valor,vencedor_cnpj,vencedor_nome,ano,mes
0,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,8,"228240,0000",09240519000111,SOFTWARE.COM.BR TECNOLOGIA E CONSULTORIA LTDA.,2024,02
1,12023,120628,GRUPAMENTO DE APOIO DE BELEM,2,Tomada de Preços,67215006797202317,52111,Comando da Aeronáutica,1206280200001202300001,MANUTENCAO / REFORMA PREDIAL,1,"955059,8100",42780861000180,RAVENA INVEST LTDA,2024,02
2,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300001,MANUTENCAO / REFORMA PREDIAL,24,"166229,7600",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02
3,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300002,SERVICO ODONTOLOGICO,12,"37385,5200",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02
4,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300003,PRESTACAO DE SERVICOS DE ELETRICISTA,12,"88556,4000",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02


In [49]:
# Criar uma nova coluna concatenada
df_itens_licitacao["chave_unica"] = (
    df_itens_licitacao["numero"].astype(str) + "_" +
    df_itens_licitacao["codigo_ug"].astype(str) + "_" +
    df_itens_licitacao["modalidade_codigo"].astype(str) + "_" +
    df_itens_licitacao["ano"].astype(str) + "_" +
    df_itens_licitacao["mes"].astype(str)
)

# Exibir o DataFrame atualizado
df_itens_licitacao.head()

,numero,codigo_ug,nome_ug,modalidade_codigo,modalidade_desc,num_processo,orgao_codigo,orgao_nome,item_codigo,item_descricao,item_qtde,item_valor,vencedor_cnpj,vencedor_nome,ano,mes,chave_unica
0,12023,110511,CENTRO GESTOR OP. SISTEMA PROTECAO AMAZONIA,5,Pregão,60090000453202210,52000,Ministério da Defesa - Unidades com vínculo,1105110500001202300001,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO D...,8,"228240,0000",09240519000111,SOFTWARE.COM.BR TECNOLOGIA E CONSULTORIA LTDA.,2024,02,12023_110511_5_2024_02
1,12023,120628,GRUPAMENTO DE APOIO DE BELEM,2,Tomada de Preços,67215006797202317,52111,Comando da Aeronáutica,1206280200001202300001,MANUTENCAO / REFORMA PREDIAL,1,"955059,8100",42780861000180,RAVENA INVEST LTDA,2024,02,12023_120628_2_2024_02
2,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300001,MANUTENCAO / REFORMA PREDIAL,24,"166229,7600",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02,12023_152711_5_2024_02
3,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300002,SERVICO ODONTOLOGICO,12,"37385,5200",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02,12023_152711_5_2024_02
4,12023,152711,INST. FED. DO RN / CAMPUS NATAL - CIDADE ALTA,5,Pregão,23466000332202317,26435,Instituto Federal do Rio Grande do Norte,1527110500001202300003,PRESTACAO DE SERVICOS DE ELETRICISTA,12,"88556,4000",13674500000150,NORTE & SUL SERVICOS TERCEIRIZADOS DE MAO DE O...,2024,02,12023_152711_5_2024_02


In [ ]:
#  Fazer o merge para trazer 'id' como 'licitacao_id' para df_participantes
df_itens_licitacao = df_itens_licitacao.merge(
    df_licitacoes_postgree[["id", "chave_unica"]],  # Apenas colunas necessárias
    on="chave_unica",  # Coluna para correspondência
    how="left"  # Mantém todas as linhas de df_participantes
)

# Renomear a coluna 'id' para 'licitacao_id'
df_itens_licitacao.rename(columns={"id": "licitacao_id"}, inplace=True)

# Exibir o DataFrame atualizado
df_itens_licitacao.head()

In [ ]:
df_itens_licitacao['item_valor'] = df_itens_licitacao['item_valor'].astype(str).str.replace(',', '.').astype(float)


In [ ]:
df_itens_licitacao_carga = df_itens_licitacao[['licitacao_id',
                                               'item_codigo',
                                               'item_descricao',
                                               'item_qtde',
                                               'item_valor',
                                               'vencedor_cnpj']].copy()

In [ ]:
# Insere dados de cnpj
resultado = inserir_dados_postgresql(df_itens_licitacao_carga, "item", DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASS, excluir_antes=True)

# Exibindo o resultado
print(resultado)

In [ ]:
# Chamar a função para carregar a tabela do PostgreSQL
df_itens_licitacao_postgree = ler_tabela_postgresql("item", DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASS)

# Exibir as 5 primeiras linhas
df_itens_licitacao_postgree.head()

### 3.6) ETL PARTICIPANTE

In [ ]:
df_participantes.head()

In [ ]:
df_participantes['item_codigo'] = df_participantes.apply(lambda row: (
    f"{str(row['codigo_ug']).zfill(6)}"
    f"{str(row['modalidade_codigo']).zfill(2)}"
    f"{str(row['numero']).zfill(5)}"
    f"{str(row['ano']).zfill(4)}"
    f"99999"
) if pd.isnull(row['item_codigo']) or len(str(row['item_codigo'])) != 22 else row['item_codigo'], axis=1)


In [ ]:
df_participantes.head()

In [ ]:
#  Fazer o merge 
df_participantes = df_participantes.merge(
    df_itens_licitacao_postgree[["id", "item_codigo"]],  # Apenas colunas necessárias
    on="item_codigo",  # Coluna para correspondência
    how="left"  # Mantém todas as linhas de df_participantes
)

# Renomear a coluna 'id' para 'item_id'
df_participantes.rename(columns={"id": "item_id"}, inplace=True)

# Exibir o DataFrame atualizado
df_participantes.head(5)

In [ ]:
df_participantes['vencedor_flag'] = df_participantes['vencedor_flag'].astype(str).str[0]
df_participantes_carga = df_participantes[['item_id', 'cnpj_codigo', 'vencedor_flag' ]]

In [ ]:
df_participantes_carga.head()

In [ ]:
# Insere dados de cnpj
resultado = inserir_dados_postgresql(df_participantes_carga, "participante", DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASS, excluir_antes=True)

# Exibindo o resultado
print(resultado)